In [1]:
import numpy as np
from PIL import Image

TOLERANCE = 1e-3
MAXDIST = 1e6

In [2]:
def length(x, y, z):
    return np.sqrt(x ** 2 + y ** 2 + z ** 2)

def unit(x, y, z):
    l = length(x, y, z)
    return (x/l, y/l, z/l)

In [3]:
class Sphere:
    def __init__(self, xc, yc, zc, r):
        self.xc = xc
        self.yc = yc
        self.zc = zc
        self.r = r
    def dist(self, x, y, z):
        dx = x - self.xc
        dy = y - self.yc
        dz = z - self.zc
        dl = length(dx, dy, dz)
        return np.maximum(0.0, dl - self.r)

class Box:
    def __init__(self, xc, yc, zc, hs):
        self.xc = xc
        self.yc = yc
        self.zc = zc
        self.hs = hs
    def dist(self, x, y, z):
        dx = np.maximum(0.0, np.abs(x - self.xc) - self.hs)
        dy = np.maximum(0.0, np.abs(y - self.yc) - self.hs)
        dz = np.maximum(0.0, np.abs(z - self.zc) - self.hs)
        dl = length(dx, dy, dz)
        return np.maximum(0.0, dl)

class RoundedBox:
    def __init__(self, xc, yc, zc, hs, power = 3):
        self.xc = xc
        self.yc = yc
        self.zc = zc
        self.hs = hs
        self.p = power
    def dist(self, x, y, z):
        dx = np.abs(x - self.xc)
        dy = np.abs(y - self.yc)
        dz = np.abs(z - self.zc)
        p = self.p
        dl = np.power(dx ** p + dy ** p + dz ** p, 1/p)
        return np.maximum(0.0, dl - self.hs)

In [4]:
# define geometry

objs = []

# add random spheres
for i in range(3):
    obj = Sphere(
        (np.random.random() - 0.5) * 5,
        (np.random.random() - 0.5) * 5,
        np.random.random() * 4 + 10,
        np.random.random() + 0.5
    )
    objs.append(obj)

# add random boxes
for i in range(3):
    obj = RoundedBox(
        (np.random.random() - 0.5) * 5,
        (np.random.random() - 0.5) * 5,
        np.random.random() * 4 + 10,
        np.random.random() + 0.5,
        power = 5
    )
    objs.append(obj)

# add something vaguely resembling ground
objs.append(Box(
    0.00, -1005.0, 0.0, 1000.0
))

In [5]:
# create camera at (x0, y0, z0) looking at (x1, y1, z1)
def create_camera(x0, y0, z0, x1, y1, z1, resolution_width, resolution_height):
    resolution_points = resolution_width * resolution_height
    # find camera unit directions
    zz_x, zz_y, zz_z = unit(x1 - x0, y1 - y0, z1 - z0)
    xx_x, xx_y, xx_z = unit(-zz_z, 0, zz_x)
    yy_x, yy_y, yy_z = unit(
        xx_y * zz_z - xx_z * zz_y,
        xx_z * zz_x - xx_x * zz_z,
        xx_x * zz_y - xx_y * zz_x
    )
    # points on camera plane
    u = np.linspace(-0.4, 0.4, resolution_width)
    v = np.linspace(-0.3, 0.3, resolution_height)
    uu, vv = np.meshgrid(u, v)
    uu = uu.flatten()
    vv = vv.flatten()
    # unit vectors in pixel direction
    dx, dy, dz = unit(
        zz_x + uu * xx_x + vv * yy_x,
        zz_y + uu * xx_y + vv * yy_y,
        zz_z + uu * xx_z + vv * yy_z
    )
    # create origin points
    x = x0 * np.ones(resolution_points)
    y = y0 * np.ones(resolution_points)
    z = z0 * np.ones(resolution_points)
    # done!
    return x, y, z, dx, dy, dz

resolution_width = 400
resolution_height = 300
resolution_points = resolution_width * resolution_height

x, y, z, dx, dy, dz = create_camera(0.0, 0.0, 0.0, 0.0, 0.0, 10.0, resolution_width, resolution_height)

In [6]:
# step a fixed number of times
for i in range(100):
    l = np.ones(x.size, dtype=float) * MAXDIST
    for obj in objs:
        l_i = obj.dist(x, y, z)
        # where smaller, keep the smaller distance
        l = np.minimum(l, l_i)
    x += l * dx
    y += l * dy
    z += l * dz

In [7]:
sun_x, sun_y, sun_z = unit(0.5, 0.3, 0.2)
sun_x *= TOLERANCE
sun_y *= TOLERANCE
sun_z *= TOLERANCE

# determine closest object

material_index = -1 * np.ones(l.shape, dtype=int)
sun = np.zeros(l.shape, dtype=float)
for i_obj in range(len(objs)):
    l_i = objs[i_obj].dist(x, y, z)
    sun_l_i = objs[i_obj].dist(x + sun_x, y + sun_y, z + sun_z)
    sun_i = np.maximum((sun_l_i - l_i) / TOLERANCE, 0.0)
    
    where = np.abs(l - l_i) < TOLERANCE
    material_index = np.where(where, i_obj, material_index)
    sun = np.where(where, sun_i, sun)

fog = np.power(0.5, length(x, y, z) / 50.0)
fog_color = np.array([200, 200, 255])
light = 0.3 + 0.7 * sun

def rand_color():
    r = np.random.random()
    g = np.random.random()
    b = np.random.random()
    mx = max(r, g, b)
    return np.array([r / mx * 255, g / mx * 255, b / mx * 255])
colors = [rand_color() for i in range(len(objs))]

r = np.ones(l.shape, dtype=int) * fog_color[0]
g = np.ones(l.shape, dtype=int) * fog_color[1]
b = np.ones(l.shape, dtype=int) * fog_color[2]

for i in range(len(objs)):
    r_i = (fog * colors[i][0] * light + (1.0 - fog) * fog_color[0]).astype(dtype=int)
    g_i = (fog * colors[i][1] * light + (1.0 - fog) * fog_color[1]).astype(dtype=int)
    b_i = (fog * colors[i][2] * light + (1.0 - fog) * fog_color[2]).astype(dtype=int)
    where = (i == material_index)
    r = np.where(where, r_i, r)
    g = np.where(where, g_i, g)
    b = np.where(where, b_i, b)

In [8]:
img = Image.new(mode = 'RGB', size = (resolution_width, resolution_height))
pix = img.load()
for j in range(resolution_height):
    for i in range(resolution_width):
        idx = j * resolution_width + i
        pix[i, resolution_height - j - 1] = (r[idx], g[idx], b[idx])
img.save('out.png')